# Contextual Deep-learning chatbot

Based on bot from [here](https://chatbotsmagazine.com/contextual-chat-bots-with-tensorflow-4391749d0077)

## How to run

Writen specially for google colab.

To train bot you need to upload intents.json file into /content directory. 
Sample of this file can be found at [github](https://github.com/AlexBSoft/Contextual-Deep-learning-chatbot)

In [0]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random


In [0]:
# import our chat-bot intents file
import json
with open('/content/intents.json') as json_data:
    intents = json.load(json_data)

In [14]:
words = []
classes = []
documents = []
ignore_words = ['?']
try:
  # loop through each sentence in our intents patterns
  for intent in intents['intents']:
      for pattern in intent['patterns']:
          # tokenize each word in the sentence
          w = nltk.word_tokenize(pattern)
          # add to our words list
          words.extend(w)
          # add to documents in our corpus
          documents.append((w, intent['tag']))
          # add to our classes list
          if intent['tag'] not in classes:
              classes.append(intent['tag'])

  # stem and lower each word and remove duplicates
  words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
  words = sorted(list(set(words)))

  # remove duplicates
  classes = sorted(list(set(classes)))

  print (len(documents), "documents")
  print (len(classes), "classes", classes)
  print (len(words), "unique stemmed words", words)
  
except LookupError:
  print ('\033[1m',"You need to manually download punkt using NLTK Downloader",'\033[0m')
  print ('\033[1m',"Type 'd' and write 'punkt'",'\033[0m')
  
  nltk.download()

45 documents
10 classes ['opentoday', 'rental', 'today', 'дела', 'имя', 'маты', 'обо', 'пока', 'привет', 'спасибо']
73 unique stemmed words ["'d", "'s", 'a', 'ar', 'can', 'do', 'doe', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'lik', 'mop', 'op', 'rent', 'thank', 'that', 'thi', 'to', 'today', 'we', 'what', 'when', 'work', 'yo', 'you', 'благодарю', 'блять', 'бот', 'вечер', 'дела', 'день', 'до', 'доброе', 'добрый', 'досвиданья', 'еблан', 'желаю', 'здравия', 'здравствуйте', 'здрасьте', 'зовут', 'имя', 'как', 'какого', 'какое', 'кто', 'нахуй', 'нового', 'отец', 'пиздец', 'пока', 'привет', 'приветствую', 'свидание', 'свидания', 'свиданья', 'создал', 'создатель', 'спасибо', 'твое', 'твой', 'тебя', 'тупой', 'ты', 'утро', 'хай', 'хуила', 'хуй', 'что']


In [0]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [0]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

tf.train.get_checkpoint_state('checkpoints')

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('/content/model.tflearn')


In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

In [0]:
print(model.predict([p]))

In [0]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

#Here it is our chatbot

For running in production you may separate first part and this part.

But don't forget to import all libraries (see the first code block)

In [0]:
# restore all of our data structures

tf.reset_default_graph()

import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [0]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

In [0]:
# load our saved model

model.load('/content/model.tflearn')

In [0]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        #return print(random.choice(i['responses']))
                        return random.choice(i['responses'])

            results.pop(0)

In [0]:
# Now some tests

# Simple chat
while 1:
  print("> ", response(input()))

In [0]:
# Classifier

while 1:
  classify(input())